# Introduction

## Autoencoders

Autoencoders are a type of artificial neural network used to learn efficient representations of data, typically for the purpose of dimensionality reduction or feature learning. They are composed of two main parts:

1. **Encoder**: This part of the network compresses the input data into a latent-space representation. It reduces the dimensionality of the data, capturing its essential features.

2. **Decoder**: This part of the network reconstructs the data from the latent-space representation, ideally producing an output that is as close to the original input as possible.

The objective of an autoencoder is to minimize the difference between the input and the reconstructed output, typically measured using a reconstruction loss such as Mean Squared Error (MSE).

## Variational Autoencoders (VAEs)

Variational Autoencoders (VAEs) are a specific type of autoencoder that not only learn to encode data into a lower-dimensional representation but also impose a probabilistic structure on this latent space. This allows for the generation of new data samples.

In addition to the encoder and decoder, VAEs introduce two more components:

1. **Latent Space Mean and Variance**: Instead of mapping inputs to a single point in the latent space, VAEs map inputs to a distribution in the latent space, characterized by a mean (μ) and variance (σ²). 

2. **Reparameterization Trick**: To enable backpropagation through the sampling process.

## Challenges

In this notebook, we will implement and train two models: a standard Autoencoder and a Variational Autoencoder (VAE). Our goals are as follows:

1. **Implement the Models**: Design and implement the architectures for both the Autoencoder and VAE.
2. **Train the Models**: Train both models on the given dataset, optimizing their respective objective functions.
3. **Evaluate Reconstruction Loss**: Measure and report the reconstruction loss for both models, comparing their performance.
4. **Visualize Reconstructed Images**: Visualize and compare the reconstructed images produced by both models.
5. **Generate New Images**: Use the trained models to generate new images from the learned latent space.

# Model Architecture

## Autoencoder

The Autoencoder architecture consists of two main parts: the encoder and the decoder.

### Encoder

The encoder compresses the input image into a lower-dimensional latent space representation. The architecture is as follows:

1. **Conv2d()**: Decreasing the dimensions of the inputs.
2. **BatchNorm2d()**: Normalizes the output of the convolutional layer to stabilize learning.
3. **LeakyReLU()**: Applies a non-linear activation function.
4. **Dropout(0.25)**: Reduces overfitting by randomly setting 25% of the input units to 0.

### Decoder

The decoder reconstructs the input image from the latent space representation. The architecture is as follows:

1. **ConvTranspose2d()**: Increasing the dimensions of the inputs.
2. **BatchNorm2d()**: Normalization.
3. **LeakyReLU()**: Activation function.
4. **Dropout(0.25)**: Reduces overfitting by randomly setting 25% of the input units to 0.




![title](images/autoencoder.png)

## Variational Autoencoder (VAE)

The VAE architecture shares the same encoder structure as the Autoencoder, with additional components for generating the latent space distribution.

### Encoder and Decoder

The Encoder and Decoder structure is identical to that of the Autoencoder, mapping the latent space representation forward and back to the original input dimensions.

### Latent Space

1. **Linear(_, 100)**: Fully connected layer to compute the mean (z_mean) and log variance (z_log_var) of the latent space distribution. 

#### log var
since we have to make sure that the variance is always positive

$$ \log(\sigma^2) = 2 \log(\sigma) $$

$$  \log(\sigma^2) = 2 \log(\sigma) $$

$$ \frac{\log(\sigma^2)}{2} = \log(\sigma) $$

$$ \sigma = e^{\frac{\log(\sigma^2)}{2}} $$


### Reparameterization

To enable backpropagation through the sampling process:

**Reparameterize(z_mu, z_log_var)**: Samples from the latent space distribution using the computed mean and log variance, incorporating Gaussian noise. Externalising the randomness ofin z by re parameterizing the variable as deterministic and differentiable function.

 1. Location-Scale Transformation $$ z = \mu + e^{\frac{\log(\sigma^2)}{2}} \cdot \epsilon $$


![title](images/vae.png)

## Evidence Lower Bound (ELBO)

In the context of Variational Autoencoders (VAEs), we aim to model the probability distribution of our data $ p(x) $ by introducing a set of latent variables $ z $. Using Bayes' rule, the posterior distribution of the latent variables given the data is expressed as:

$$
p(z|x) = \frac{p(x|z) p(z)}{p(x)}
$$

However, calculating the denominator \( p(x) \) involves integrating over all possible values of $ z $, leading to the following intractable integral:

$$
p(x) = \int p(x|z) p(z) \, dz
$$

This integral is often intractable in high-dimensional spaces because it requires summing over an exponential number of possible configurations of $ z $. 

### Solution: Variational Inference

ELBO is used as loss function for variational inference

To address this, we use **Variational Inference** to approximate the intractable posterior $p(z|x)$ with another distribution $ q_\phi(z|x) $, parameterized by $ \phi $. The goal is to make $ q_\phi(z|x) $ as close as possible to the true posterior $ p(z|x) $.


The ELBO provides a tractable lower bound on the log likelihood of the data $ \log p(x) $. It is derived as follows:
marginal log likelihood
$$
\log p(x) \geq \mathbb{E}_{q_\phi(z|x)} \left[ \log p(x|z) \right] - \text{KL}(q_\phi(z|x) \| p(z))
$$

Here, the ELBO consists of two terms:

1. **Reconstruction Loss**: $ \mathbb{E}_{q_\phi(z|x)} \left[ \log p(x|z) \right] $
   - This term measures how well the VAE reconstructs the input data $  x $ from the latent variable $ z $.

2. **KL Divergence**: $ \text{KL}(q_\phi(z|x) \| p(z)) $
   - This term measures the divergence between the approximate posterior $ q_\phi(z|x) $ and the prior $ p(z) $. It acts as a regularizer, encouraging $ q_\phi(z|x) $ to be close to the prior $ p(z) $.

### Objective: Maximize the ELBO

To train the VAE, we maximize the ELBO with respect to the parameters $ \theta $ of the decoder and $ \phi $ of the encoder. Maximizing the ELBO is equivalent to maximizing a lower bound on the log likelihood $ \log p(x) $, making it possible to effectively train the model despite the intractable integral.



# Loss Functions

In Autoencoder the loss function is L2 or MSE between the features and the logits. The loss function for a VAE consists of two main components: the reconstruction loss and the KL divergence loss.

## Reconstruction Loss

The reconstruction loss measures how well the decoder can reconstruct the input from the latent space representation. It quantifies the difference between the original input and the reconstructed output. Common choices for the reconstruction loss include:

- **Mean Squared Error (MSE)**: 
$$ \text{MSE} = \frac{1}{N} \sum_{i=1}^{N} (x_i - \hat{x}_i)^2 $$

Reconstruction los for AE


![title](images/autoencoder_loss.png)

Reconstruction loss for VAE


![title](images/ReconstructionLoss_vae.png)

## KL Divergence Loss

The KL divergence loss measures the difference between the learned latent distribution and the prior distribution (typically a standard normal distribution). It ensures that the latent space follows the desired distribution.The KL divergence for a Gaussian can be computed as:

$$ \text{KL}(q(z|x) \| p(z)) = -\frac{1}{2} \sum_{1}^{N} (1 + \log(\sigma^2) - \mu^2 - \sigma^2) $$

where $\mu $ and $\sigma$ are the mean and standard deviation of the latent distribution, respectively.


![title](images/KL_vae.png)



## Combined Loss

The total loss for a VAE is the sum of the reconstruction loss and the KL divergence loss:

$$ \text{Total Loss} = \text{Reconstruction Loss} + \text{KL Divergence Loss} $$

This ensures that the model learns to accurately reconstruct the input while regularizing the latent space to follow the desired distribution.

![title](images/combinedloss_vae.png)

# Outcomes

## Autoencoder (AE)

- **Reconstruction Quality**:
  - The AE did a good job on reconstructing images while doing not so well in generation, implicating it is more useful for dimensionality tasks.

## Variational Autoencoder (VAE)

- **Generative Capabilities**:
  - In contrast, the VAE was better on generating images while doing not so well in reconstruction.
  

# Conclusion

## Self-Evaluation

1. **Comprehensive Architecture Design**:
   - Both the Autoencoder (AE) and the Variational Autoencoder (VAE) architectures were designed with a clear understanding.
   
2. **Effective Use of Loss Functions**:
   - The loss functions for both models are well-chosen and effectively implemented. The use of Mean Squared Error (MSE) for the AE and the combination of reconstruction loss with KL divergence for the VAE ensures that each model is optimized.

3. **Clear Differentiation of Model Capabilities**:
   - The outcomes section effectively highlights the distinct advantages of AEs and VAEs, making it clear that AEs are best suited for reconstruction while VAEs excel in generative tasks. This clear differentiation helps in understanding the application scenarios for each model.

4. **Visual Demonstrations**:
   - Clear visualizations of reconstructed images and newly generated samples provides a qualitative assessment of model performance, which is crucial for understanding how well the models work beyond mere numerical metrics.

### Areas for Improvement

1. **Enhanced Hyperparameter Tuning**:
   - More extensive hyperparameter tuning could further improve the performance of both models.

2. **Advanced Variational Techniques**:
   - The VAE implementation could benefit from more advanced variational techniques, such as employing different priors for specific tasks.
